# L1 — Model, Prompts, and Output Parsing (LangChain v1 + OpenAI Responses)

# Setup

This notebook uses **OpenAI (Python SDK v2) + LangChain v1**.

## Prereqs
1. Set your API key in the environment:

```bash
export OPENAI_API_KEY="..."
```

2. Restart the kernel after setting env vars.


In [1]:
import os

# Make sure your key is set
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your environment before running."

MODEL = "gpt-5-mini"


## 1) Direct OpenAI call (Responses API)

In [2]:
from openai import OpenAI

client = OpenAI()

resp = client.responses.create(
    model=MODEL,
    input="What is 1+1? Answer with just the number."
)

print(resp.output_text)


2


## 2) LangChain ChatOpenAI + Prompt Templates

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model=MODEL)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{question}")
])

chain = prompt | llm

msg = chain.invoke({"question": "Explain 1+1 in one sentence."})
print(msg.content)


1 + 1 = 2 because adding one unit to another unit results in two units.


## 3) Structured output with Pydantic

In [5]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class MathAnswer(BaseModel):
    answer: int = Field(..., description="The numeric answer.")
    explanation: str = Field(..., description="Short explanation.")

parser = PydanticOutputParser(pydantic_object=MathAnswer)
print(f'parser is an object of type: {type(parser)}')

prompt = ChatPromptTemplate.from_messages([
    ("system", "Return JSON only. Follow the format instructions strictly.{format_instructions}"),
    ("user", "Question: {question}")
]).partial(format_instructions=parser.get_format_instructions())
print(f'prompt is an object of type: {type(prompt)}')

chain = prompt | llm | parser
print(f'chain is an object of type: {type(chain)}')

out = chain.invoke({"question": "What is 1+1?"})
print(f'out is an object of type: {type(out)}')
print(out)
print(out.answer, out.explanation)


parser is an object of type: <class 'langchain_core.output_parsers.pydantic.PydanticOutputParser'>
prompt is an object of type: <class 'langchain_core.prompts.chat.ChatPromptTemplate'>
chain is an object of type: <class 'langchain_core.runnables.base.RunnableSequence'>
out is an object of type: <class '__main__.MathAnswer'>
answer=2 explanation='1 plus 1 equals 2.'
2 1 plus 1 equals 2.


## 4) Tool-free 'prompt parser' pattern (robust extraction)
If the model returns extra text, parse with a schema and retry.

In [6]:
from langchain_core.runnables import RunnableLambda

def robust_parse(question: str, max_tries: int = 2):
    last_err = None
    for _ in range(max_tries):
        try:
            return chain.invoke({"question": question})
        except Exception as e:
            last_err = e
    raise last_err

print(robust_parse("What is 7+5?"))


answer=12 explanation='7 + 5 = 12.'


## 5) Best of Both Worlds - Structured Output + Retries + "Automatic Fix the JSON and Retry” Behavior When Parsing Fails.

In [13]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_classic.output_parsers import OutputFixingParser
from langchain_core.exceptions import OutputParserException

# 1) Define the schema you want back
class MathAnswer(BaseModel):
    answer: int = Field(..., description="The numeric answer.")
    explanation: str = Field(..., description="Short explanation.")

# 2) Build an LLM
llm = ChatOpenAI(model=MODEL, temperature=0)

# 3) Base parser (strict)
base_parser = PydanticOutputParser(pydantic_object=MathAnswer)

# 4) Fixing parser (if parsing fails, it uses the LLM to fix the output)
fixing_parser = OutputFixingParser.from_llm(llm=llm, parser=base_parser)

# 5) Prompt that tells the model to output JSON matching the schema
prompt = ChatPromptTemplate.from_messages([
    ("system", "Return JSON only. Follow the format instructions strictly.\n{format_instructions}"),
    ("user", "Question: {question}")
]).partial(format_instructions=base_parser.get_format_instructions())

# 6) IMPORTANT: call the LLM first, then parse with "try strict, then fix"
llm_chain = prompt | llm

def invoke_structured(question: str, max_tries: int = 3) -> MathAnswer:
    last_err = None

    for _ in range(max_tries):
        try:
            msg = llm_chain.invoke({"question": question})
            text = msg.content

            # Try strict parsing first
            try:
                return base_parser.parse(text)
            except OutputParserException:
                # If it's not valid JSON / wrong schema, ask LLM to "fix" it and re-parse
                return fixing_parser.parse(text)

        except Exception as e:
            last_err = e

    raise last_err

# Example
out = invoke_structured("What is 7+5?")
print(out)                 # Pydantic model repr (not JSON)
print(out.answer)          # 12
print(out.explanation)     # short explanation


answer=12 explanation='7 + 5 equals 12 because adding seven and five yields twelve.'
12
7 + 5 equals 12 because adding seven and five yields twelve.
